In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# my_drive = GoogleDrive(gauth)

In [4]:
# def clear_trash():
#   for a_file in my_drive.ListFile({'q': "trashed = true"}).GetList():
#     # print the name of the file being deleted.
#     try:
#       print(f"the file {a_file['title']}, is about to get deleted permanently.")
#       # delete the file permanently.
#       a_file.Delete()
#     except:
#       pass

In [5]:
# clear_trash()

In [9]:
# !cp /content/drive/MyDrive/TalkLikeMom/conversations-main.zip /content
# !unzip conversations-main.zip

In [8]:
cd /content/conversations-main/Classifier/

/content/conversations-main/Classifier


In [7]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [10]:
import torch
torch.cuda.is_available()

True

In [11]:
import sys
sys.path.append('.')

In [12]:
cp ../utils.py .

In [13]:
from utils import load_jsonl, dump_jsonl, set_random_seed

In [14]:
from data_loader import get_task1_conver, get_task2_conver, preprocess

In [15]:
import pandas as pd

In [16]:
df = get_task1_conver("../Task1//annotated_conersations.jsonl", "closeness", skips = ["4. Don't like each other"], only_user=False)
# print(df[0]["text"][0])
pd.concat(df).groupby("label").count()

Loaded 1234 records from ../Task1//annotated_conersations.jsonl
N 1096 60 60


,text
label,
1. Close,551
2. Know each other,230
3. Don't know each other,435


In [17]:

df = get_task2_conver("../Task2/annotated/annotated.jsonl", "closeness", skips = [], only_user=False)
# print(df[0]["text"][0])
pd.concat(df).groupby("label").count()

Loaded 2486 records from ../Task2/annotated/annotated.jsonl
N 1700 212 212


,text
label,
1. Close,222
2. Know each other,158
3. Don't know each other,1487
4. Don't like each other,257


In [18]:
df = get_task1_conver("../Task3/annotated/annotated.jsonl", "closeness", skips = [], only_user=False)
# print(df[0]["text"][0])
pd.concat(df).groupby("label").count()

Loaded 1221 records from ../Task3/annotated/annotated.jsonl
N 1099 61 61


,text
label,
1. Close,462
2. Know each other,696
3. Don't know each other,52
4. Don't like each other,11


In [19]:
!pip install -q transformers pythainlp datasets evaluate sentencepiece
# !pip install -q accelerate -U
!pip install -q transformers[torch]
# !pip install -q
!pip install -q shap nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 538.2/538.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.4 MB/s eta 0:00:00


In [20]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# import wandb
# from sklearn.metrics import precision_recall_fscore_support, classification_report
# from pythainlp.tokenize import word_tokenize
import torch
import datasets
from datasets import Dataset, DatasetDict
from transformers import DataCollatorWithPadding, DefaultDataCollator
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer



In [21]:
import random
def set_random_seed(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    np.random.RandomState(seed)

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) #seed all gpus
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.enabled = False
    torch.backends.cudnn.benchmark = False

In [22]:
from sklearn.utils import compute_class_weight
import torch.nn as nn
import os, shutil
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import mean_squared_error, r2_score
import json

def run_exp(out_dir, df, regressor_configs, seed=42, report="none", model_name = "facebookaI/xlm-roberta-base"):

    set_random_seed(seed)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # device = torch.device("cpu")
    print("START")
    print("step 1: load data")
    train, val, test = df

    # train = train.head(100)
    # val = val.head(100)
    # test = test.head(100)

    print("step 2: load tokenizer")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    num_added_toks = tokenizer.add_special_tokens({"additional_special_tokens": ["usr", "sys", "rep"]})

    print("step 3: init data")
    ds = DatasetDict()
    ds['train'] = Dataset.from_pandas(train)
    ds['val'] = Dataset.from_pandas(val)
    ds['test'] = Dataset.from_pandas(test)

    num_labels = 1
    id2label = {1: regressor_configs["label"]}
    label2id = [regressor_configs["label"]]

    def word_tokenize(d, tokenizer=None, label2id=None, max_length=256):
        texts = [preprocess(t) for t in d["text"]]
#         print(texts)
        tokens = tokenizer(texts, padding='max_length', truncation=True, max_length=max_length)
        tokens["label"] = [regressor_configs["label_fn"](label) for label in d["label"]]
        return tokens

    tokenized_ds = ds.map(word_tokenize, batched=True, fn_kwargs={"tokenizer":tokenizer, "label2id": label2id, "max_length":max_length})
    data_collator = DefaultDataCollator()

    print("step 4: load model")
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, id2label=id2label, label2id=label2id);
    model.resize_token_embeddings(len(tokenizer)+1)
    model = model.to(device)

    # print(model)

    if regressor_configs is None:
        metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])
        def compute_metrics(eval_pred):
            predictions, labels = eval_pred
            predictions = np.argmax(predictions, axis=1)
            return metrics.compute(predictions=predictions, references=labels, average="macro")
    else:
        label_fn = regressor_configs["label_fn"]
        def compute_metrics(eval_pred):
            predictions, actual = eval_pred
            predictions = predictions.reshape(-1)

            predicted_labels = [label_fn(p) for p in predictions]
            actual_labels = [label_fn(p) for p in actual]
            p, r, f1, _ = precision_recall_fscore_support(actual_labels, predicted_labels, average='macro')

            return {
                "r2_score": r2_score(actual, predictions),
                "mean_squared_error": np.sqrt(mean_squared_error(actual, predictions)),
                "accuracy": accuracy_score(actual_labels, predicted_labels),
                "f1": f1,
                "precision": p,
                "recall": r,
            }


    print("step 5: fine-tune")

    training_args = TrainingArguments(
        output_dir=out_dir,
        warmup_ratio=0.1,
        learning_rate=1e-4,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        weight_decay=0.01,
        logging_strategy="epoch",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        report_to=report,
        metric_for_best_model="f1",
        save_total_limit=2,
        load_best_model_at_end=True,
        push_to_hub=False,
        run_name=out_dir,
    )

    # print(training_args)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_ds["train"],
        eval_dataset=tokenized_ds["val"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    best_ckpt_path = trainer.state.best_model_checkpoint
    print(best_ckpt_path)

    modle_out_path = out_dir+"/best_model"
    if os.path.exists(modle_out_path):
        shutil.rmtree(modle_out_path)

    os.rename(best_ckpt_path, modle_out_path)
    best_ckpt_path = modle_out_path

    for split, d in [("train", train), ("val", val), ("test", test)]:
      o = trainer.predict(tokenized_ds[split])
      predictions = o.predictions.reshape(-1).tolist()
      d["predict"] = predictions


    train.to_csv(modle_out_path+"/train.csv", index=False)
    val.to_csv(modle_out_path+"/val.csv", index=False)
    test.to_csv(modle_out_path+"/test.csv", index=False)

    print("step 6: evaluate")
    eval_train = trainer.evaluate(tokenized_ds["train"])
    eval_test = trainer.evaluate(tokenized_ds["test"])
    eval_val = trainer.evaluate(tokenized_ds["val"])
    print(eval_test)

    with open(f"{modle_out_path}/eval.json", 'w') as fout:
      json.dump({"train": eval_train, "test": eval_test, "val": eval_val}, fout)

    print("DONE")

    return trainer, eval_val, eval_test

## Task1: Train Model

In [23]:
report = "none"
batch_size = 64
max_length = 128
num_epochs = 20

In [24]:
# df[0]["label"].value_counts().loc[['1. Close', '2. Know each other', "3. Don't know each other"]].plot.bar()

In [25]:
# df[1]["label"].value_counts().loc[['1. Close', '2. Know each other', "3. Don't know each other"]].plot.bar()

In [26]:
# df[2]["label"].value_counts().loc[['1. Close', '2. Know each other', "3. Don't know each other"]].plot.bar()

In [27]:
def closeness_label_fn(label):
    if label == '1. Close':
        return 1.0
    elif label =='2. Know each other':
        return 0.0
    elif label == "3. Don't know each other":
        return -1.0
    elif type(label)==str:
        assert(False)

    if label > 0.5:
        return '1. Close'
    elif label < -0.5:
        return "3. Don't know each other"
    else:
        return '2. Know each other'

In [28]:
import os, shutil
def copytree(src, dst, symlinks=False, ignore=None):

  if os.path.isdir(src):
    if not os.path.exists(dst):
        os.makedirs(dst)

    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)
  else:
      dst_dir = "/".join(dst.split("/")[0:-1])
      if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)
      shutil.copy2(src, dst)

In [29]:
import shap
import scipy as sp

import pickle

def load_shap_values(filepath):
    with open(filepath, 'rb') as fin:
        obj = pickle.load(fin)
    return obj

def save_shap_values(filepath, obj):
    with open(filepath, 'wb') as fin:
        pickle.dump(obj, fin)


def get_shapley(df, trainer, shap_path):
    train, val, test = df
    train["split"] = "train"
    val["split"] = "val"
    test["split"] = "test"

    df = pd.concat([train, test, val])
    print("DATA SIZE", len(df))
    def f(x):
        tokenizer = trainer.tokenizer
        model = trainer.model

        # print(type(x.tolist()))
        tv = tokenizer(x.tolist(), padding='max_length', truncation=True, max_length=max_length, return_tensors="pt")
        input_ids = tv["input_ids"].cuda()
        attention_mask = tv["attention_mask"].cuda()
        # tv = tokenizer(x, truncation=True, max_length=max_length).cuda()
        outputs = model(input_ids, attention_mask=attention_mask)[0].detach().cpu().numpy()

        # scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
        # val = sp.special.logit(scores[:,1]) # use one vs rest logit units

        val = outputs.reshape(-1)
        return val

    explainer = shap.Explainer(f, trainer.tokenizer)
    data = {
        "label": df["label"].values,
        "text": [preprocess(t) for t in df["text"].values],
    }

    shap_values = explainer(data)

    save_shap_values(shap_path, {
        "values": shap_values,
        "inputs": data
    })

    return shap_values

In [26]:
best_trainer = None
best_eval_val = None
best_eval_test = None
best_trainer_idx = None

task_name = "xl_task1_clse_usr"
for i in range(5):
  df = get_task1_conver("../Task1/annotated_conersations.jsonl", "closeness", skips = ["4. Don't like each other"], only_user=True)
  trainer, eval_val, eval_test = run_exp(f"./Regressors/{task_name}{i}", df, seed=i, report=report, regressor_configs={
      "label": "close",
      "not_label": "not_close",
      "label_fn": closeness_label_fn,
  })

  print(f"==== round {i+1} ====")
  print("Result[val]", eval_val)
  print("Result[test]", eval_test)

  if abs(eval_val["eval_f1"] - eval_test["eval_f1"]) > 0.15:
    continue

  if (best_eval_val is None) or (best_eval_val["eval_f1"] < eval_val["eval_f1"]):
    best_eval_val = eval_val
    best_eval_test = eval_test
    best_trainer = trainer
    best_trainer_idx = i

  del trainer

print()
print()
print("===================================")
print()
print("BEST MODEL")
print("Result[val]", best_eval_val)
print("Result[test]", best_eval_test)


Loaded 1234 records from ../Task1/annotated_conersations.jsonl
N 1096 60 60
START
step 1: load data
step 2: load tokenizer


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


step 3: init data


Map:   0%|          | 0/1096 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

...

step 5: fine-tune


Epoch,Training Loss,Validation Loss,R2 Score,Mean Squared Error,Accuracy,F1,Precision,Recall
1,0.712700,0.608925,0.277717,0.780336,0.400000,0.359924,0.584615,0.371463
2,0.492700,0.610281,0.276108,0.781205,0.366667,0.338762,0.526190,0.370198
3,0.436800,0.442607,0.474997,0.665287,0.583333,0.522027,0.547222,0.510927
4,0.393700,0.608943,0.277695,0.780348,0.500000,0.399824,0.617424,0.440304
5,0.351300,0.539305,0.360297,0.734374,0.600000,0.573365,0.632897,0.628537
6,0.359900,0.535717,0.364553,0.731927,0.533333,0.498446,0.677778,0.608638
7,0.293200,0.519048,0.384325,0.720450,0.666667,0.593333,0.595398,0.603347
8,0.197900,0.395018,0.531444,0.628505,0.666667,0.580247,0.593407,0.573039
9,0.197300,0.536968,0.363069,0.732781,0.700000,0.566380,0.582011,0.566540
10,0.165200,0.384338,0.544114,0.619950,0.683333,0.610750,0.625169,0.607488


./Regressors/xl_task1_clse_usr4/checkpoint-360


step 6: evaluate


{'eval_loss': 0.17371687293052673, 'eval_r2_score': 0.7872130742926304, 'eval_mean_squared_error': 0.4167936146259308, 'eval_accuracy': 0.8, 'eval_f1': 0.6687500000000001, 'eval_precision': 0.6715007215007215, 'eval_recall': 0.6692353643966547, 'eval_runtime': 0.3713, 'eval_samples_per_second': 161.593, 'eval_steps_per_second': 2.693, 'epoch': 20.0}
DONE
==== round 5 ====
Result[val] {'eval_loss': 0.3999297320842743, 'eval_r2_score': 0.525618666439227, 'eval_mean_squared_error': 0.6324000954627991, 'eval_accuracy': 0.7, 'eval_f1': 0.6227148330596607, 'eval_precision': 0.633599149388623, 'eval_recall': 0.6193926846100759, 'eval_runtime': 0.3763, 'eval_samples_per_second': 159.447, 'eval_steps_per_second': 2.657, 'epoch': 20.0}
Result[test] {'eval_loss': 0.17371687293052673, 'eval_r2_score': 0.7872130742926304, 'eval_mean_squared_error': 0.4167936146259308, 'eval_accuracy': 0.8, 'eval_f1': 0.6687500000000001, 'eval_precision': 0.6715007215007215, 'eval_recall': 0.6692353643966547, 'eval_

In [27]:

for i in range(5):
  src = f"./Regressors/{task_name}{i}/best_model"
  tgt = f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/Regressors/{task_name}{i}"

  if i==best_trainer_idx:
    copytree(f"{src}/train.csv", f"{tgt}/train.csv", symlinks=False, ignore=None)
    copytree(f"{src}/val.csv", f"{tgt}/val.csv", symlinks=False, ignore=None)
    copytree(f"{src}/test.csv", f"{tgt}/test.csv", symlinks=False, ignore=None)
    copytree(f"{src}/eval.json", f"{tgt}/eval.json", symlinks=False, ignore=None)

    tgt = f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/Regressors/{task_name}_best"
    copytree(src, tgt, symlinks=False, ignore=None)
  else:
    copytree(f"{src}/train.csv", f"{tgt}/train.csv", symlinks=False, ignore=None)
    copytree(f"{src}/val.csv", f"{tgt}/val.csv", symlinks=False, ignore=None)
    copytree(f"{src}/test.csv", f"{tgt}/test.csv", symlinks=False, ignore=None)
    copytree(f"{src}/eval.json", f"{tgt}/eval.json", symlinks=False, ignore=None)

  print(f"SAVED {i}")

SAVED 0
SAVED 1
SAVED 2
SAVED 3
SAVED 4


In [28]:
get_shapley(df, best_trainer, f"./ShapleyValues/{task_name}.pkl");
copytree(f"./ShapleyValues/{task_name}.pkl", f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/ShapleyValues/{task_name}.pkl", symlinks=False, ignore=None)

DATA SIZE 1216


PartitionExplainer explainer: 1217it [1:13:58,  3.66s/it]


In [30]:
def authority1_label_fn(label):
    if label == '0. Very respect':
        return 1.0
    elif label =='1. Respect':
        return 0.0
    elif label == "2. Normal":
        return -1.0

    elif type(label)==str:
        assert(False)

    # [0, 0.33) =>
    # [0.33, 0.66) =>
    # [0.66, 1] =>

    if label > 0.5:
        return '0. Very respect'
    elif label < -0.5:
        return "2. Normal"
    else:
        return '1. Respect'

In [30]:
best_trainer = None
best_eval_val = None
best_eval_test = None
best_trainer_idx = None

task_name = "xl_task1_auth_usr"
for i in range(5):
  df = get_task1_conver("../Task1/annotated_conersations.jsonl", "authority", skips = ["3. Not respect"], only_user=True)
  trainer, eval_val, eval_test = run_exp(f"./Regressors/{task_name}{i}", df, seed=i, report=report, regressor_configs={
      "label": "respect",
      "not_label": "not_respect",
      "label_fn": authority1_label_fn,
  })

  print(f"==== round {i+1} ====")
  print("Result[val]", eval_val)
  print("Result[test]", eval_test)

  if abs(eval_val["eval_f1"] - eval_test["eval_f1"]) > 0.15:
    continue

  if (best_eval_val is None) or (best_eval_val["eval_f1"] < eval_val["eval_f1"]):
    best_eval_val = eval_val
    best_eval_test = eval_test
    best_trainer = trainer
    best_trainer_idx = i

  del trainer

print()
print()
print("===================================")
print()
print("BEST MODEL")
print("Result[val]", best_eval_val)
print("Result[test]", best_eval_test)

Loaded 1234 records from ../Task1/annotated_conersations.jsonl
N 1098 61 61
START
step 1: load data
step 2: load tokenizer
step 3: init data


Map:   0%|          | 0/1098 [00:00<?, ? examples/s]

Map:   0%|          | 0/61 [00:00<?, ? examples/s]

Map:   0%|          | 0/61 [00:00<?, ? examples/s]

step 4: load model


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


step 6: evaluate


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.7464497089385986, 'eval_r2_score': -0.07740098324136446, 'eval_mean_squared_error': 0.8639732599258423, 'eval_accuracy': 0.5245901639344263, 'eval_f1': 0.2293906810035842, 'eval_precision': 0.17486338797814208, 'eval_recall': 0.3333333333333333, 'eval_runtime': 0.4267, 'eval_samples_per_second': 142.969, 'eval_steps_per_second': 2.344, 'epoch': 20.0}
DONE
==== round 5 ====
Result[val] {'eval_loss': 0.6464042663574219, 'eval_r2_score': -0.035861538770629586, 'eval_mean_squared_error': 0.803992748260498, 'eval_accuracy': 0.5573770491803278, 'eval_f1': 0.23859649122807017, 'eval_precision': 0.18579234972677594, 'eval_recall': 0.3333333333333333, 'eval_runtime': 0.4412, 'eval_samples_per_second': 138.261, 'eval_steps_per_second': 2.267, 'epoch': 20.0}
Result[test] {'eval_loss': 0.7464497089385986, 'eval_r2_score': -0.07740098324136446, 'eval_mean_squared_error': 0.8639732599258423, 'eval_accuracy': 0.5245901639344263, 'eval_f1': 0.2293906810035842, 'eval_precision': 0.17486

In [31]:
for i in range(5):
  src = f"./Regressors/{task_name}{i}/best_model"
  tgt = f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/Regressors/{task_name}{i}"

  if i==best_trainer_idx:
    copytree(f"{src}/train.csv", f"{tgt}/train.csv", symlinks=False, ignore=None)
    copytree(f"{src}/val.csv", f"{tgt}/val.csv", symlinks=False, ignore=None)
    copytree(f"{src}/test.csv", f"{tgt}/test.csv", symlinks=False, ignore=None)
    copytree(f"{src}/eval.json", f"{tgt}/eval.json", symlinks=False, ignore=None)

    tgt = f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/Regressors/{task_name}_best"
    copytree(src, tgt, symlinks=False, ignore=None)
  else:
    copytree(f"{src}/train.csv", f"{tgt}/train.csv", symlinks=False, ignore=None)
    copytree(f"{src}/val.csv", f"{tgt}/val.csv", symlinks=False, ignore=None)
    copytree(f"{src}/test.csv", f"{tgt}/test.csv", symlinks=False, ignore=None)
    copytree(f"{src}/eval.json", f"{tgt}/eval.json", symlinks=False, ignore=None)

  print(f"SAVED {i}")


SAVED 0
SAVED 1
SAVED 2
SAVED 3
SAVED 4


In [32]:
get_shapley(df, best_trainer, f"./ShapleyValues/{task_name}.pkl");
copytree(f"./ShapleyValues/{task_name}.pkl", f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/ShapleyValues/{task_name}.pkl", symlinks=False, ignore=None)

DATA SIZE 1220


PartitionExplainer explainer: 1221it [1:13:52,  3.64s/it]


In [33]:
!rm -rf ./Regressors/*

## Task2: Train Model

In [34]:
best_trainer = None
best_eval_val = None
best_eval_test = None
best_trainer_idx = None

_f1 = []
task_name = "xl_task2_clas_usr"
for i in range(5):
  df = get_task2_conver("../Task2/annotated/annotated.jsonl", "closeness", seed=i, skips = ["4. Don't like each other"], only_user=True)
  trainer, eval_val, eval_test = run_exp(f"./Regressors/{task_name}{i}", df, seed=i, report=report, regressor_configs={
      "label": "close",
      "not_label": "not_close",
      "label_fn": closeness_label_fn,
  })

  print(f"==== round {i+1} ====")
  print("Result[val]", eval_val)
  print("Result[test]", eval_test)

  if abs(eval_val["eval_f1"] - eval_test["eval_f1"]) > 0.15:
    continue

  if (best_eval_val is None) or (best_eval_val["eval_f1"] < eval_val["eval_f1"]):
    best_eval_val = eval_val
    best_eval_test = eval_test
    best_trainer = trainer
    best_trainer_idx = i
    _f1.append(eval_test["eval_f1"])

  del trainer

print()
print()
print("===================================")
print()
print("BEST MODEL")
print("Result[val]", best_eval_val)
print("Result[test]", best_eval_test, sum(_f1)/len(_f1))


Loaded 2486 records from ../Task2/annotated/annotated.jsonl
N 1495 186 186
START
step 1: load data
step 2: load tokenizer
step 3: init data


Map:   0%|          | 0/1495 [00:00<?, ? examples/s]

Map:   0%|          | 0/186 [00:00<?, ? examples/s]

Map:   0%|          | 0/186 [00:00<?, ? examples/s]

step 4: load model


...

Epoch,Training Loss,Validation Loss,R2 Score,Mean Squared Error,Accuracy,F1,Precision,Recall
1,0.695400,0.481892,-0.042491,0.694184,0.774194,0.290909,0.258065,0.333333
2,0.489000,0.466156,-0.008450,0.682756,0.774194,0.290909,0.258065,0.333333
3,0.476000,0.487843,-0.055366,0.698457,0.483871,0.265484,0.276801,0.308796
4,0.487600,0.451628,0.022980,0.672032,0.747312,0.286008,0.257407,0.321759
5,0.467100,0.454237,0.017335,0.673971,0.725806,0.283019,0.258621,0.312500
6,0.457400,0.526009,-0.137932,0.725265,0.704301,0.336480,0.315865,0.360648
7,0.454800,0.458901,0.007245,0.677422,0.731183,0.336022,0.317068,0.357870
8,0.440600,0.452285,0.021557,0.672522,0.419355,0.275325,0.364156,0.424537
9,0.433100,0.419755,0.091931,0.647885,0.575269,0.341503,0.354987,0.448611
10,0.381000,0.530324,-0.147267,0.728234,0.763441,0.291282,0.261510,0.328704


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

./Regressors/xl_task2_clas_usr4/checkpoint-312


step 6: evaluate


{'eval_loss': 0.3727474510669708, 'eval_r2_score': 0.1574275321351779, 'eval_mean_squared_error': 0.6105304956436157, 'eval_accuracy': 0.7849462365591398, 'eval_f1': 0.4599156118143459, 'eval_precision': 0.5744949494949495, 'eval_recall': 0.4316438287026522, 'eval_runtime': 1.2274, 'eval_samples_per_second': 151.536, 'eval_steps_per_second': 2.444, 'epoch': 20.0}
DONE
==== round 5 ====
Result[val] {'eval_loss': 0.4163185954093933, 'eval_r2_score': 0.09936482709704164, 'eval_mean_squared_error': 0.6452275514602661, 'eval_accuracy': 0.7419354838709677, 'eval_f1': 0.39988958410011044, 'eval_precision': 0.5573622402890696, 'eval_recall': 0.38665824915824915, 'eval_runtime': 1.2329, 'eval_samples_per_second': 150.862, 'eval_steps_per_second': 2.433, 'epoch': 20.0}
Result[test] {'eval_loss': 0.3727474510669708, 'eval_r2_score': 0.1574275321351779, 'eval_mean_squared_error': 0.6105304956436157, 'eval_accuracy': 0.7849462365591398, 'eval_f1': 0.4599156118143459, 'eval_precision': 0.57449494949

In [35]:
for i in range(5):
  src = f"./Regressors/{task_name}{i}/best_model"
  tgt = f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/Regressors/{task_name}{i}"

  if i==best_trainer_idx:
    copytree(f"{src}/train.csv", f"{tgt}/train.csv", symlinks=False, ignore=None)
    copytree(f"{src}/val.csv", f"{tgt}/val.csv", symlinks=False, ignore=None)
    copytree(f"{src}/test.csv", f"{tgt}/test.csv", symlinks=False, ignore=None)
    copytree(f"{src}/eval.json", f"{tgt}/eval.json", symlinks=False, ignore=None)

    tgt = f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/Regressors/{task_name}_best"
    copytree(src, tgt, symlinks=False, ignore=None)
  else:
    copytree(f"{src}/train.csv", f"{tgt}/train.csv", symlinks=False, ignore=None)
    copytree(f"{src}/val.csv", f"{tgt}/val.csv", symlinks=False, ignore=None)
    copytree(f"{src}/test.csv", f"{tgt}/test.csv", symlinks=False, ignore=None)
    copytree(f"{src}/eval.json", f"{tgt}/eval.json", symlinks=False, ignore=None)

  print(f"SAVED {i}")

SAVED 0
SAVED 1
SAVED 2
SAVED 3
SAVED 4


In [36]:
get_shapley(df, best_trainer, f"./ShapleyValues/{task_name}.pkl");
copytree(f"./ShapleyValues/{task_name}.pkl", f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/ShapleyValues/{task_name}.pkl", symlinks=False, ignore=None)

DATA SIZE 1867


PartitionExplainer explainer: 1868it [1:23:49,  2.70s/it]


In [37]:
del _f1

In [38]:
del best_trainer

In [31]:
def authority2_label_fn(label):
    if label == '1. Respect':
        return 1.0
    elif label =='2. Normal':
        return 0.0
    elif label == "3. Not respect":
        return -1.0
    elif type(label)==str:
        assert(False)

    if label > 0.5:
        return '1. Respect'
    elif label < -0.5:
        return "3. Not respect"
    else:
        return '2. Normal'

In [40]:
best_trainer = None
best_eval_val = None
best_eval_test = None
best_trainer_idx = None

task_name = "xl_task2_auth_usr"
for i in range(5):
  df = get_task2_conver("../Task2/annotated/annotated.jsonl", "authority", seed=i, skips = [], only_user=True)
  trainer, eval_val, eval_test = run_exp(f"./Regressors/{task_name}{i}", df, seed=i, report=report, regressor_configs={
      "label": "respect",
      "not_label": "not_respect",
      "label_fn": authority2_label_fn,
  })

  print(f"==== round {i+1} ====")
  print("Result[val]", eval_val)
  print("Result[test]", eval_test)

  if abs(eval_val["eval_f1"] - eval_test["eval_f1"]) > 0.15:
    continue

  if (best_eval_val is None) or (best_eval_val["eval_f1"] < eval_val["eval_f1"]):
    best_eval_val = eval_val
    best_eval_test = eval_test
    best_trainer = trainer
    best_trainer_idx = i

  del trainer

print()
print()
print("===================================")
print()
print("BEST MODEL")
print("Result[val]", best_eval_val)
print("Result[test]", best_eval_test)


Loaded 2486 records from ../Task2/annotated/annotated.jsonl
N 1876 234 234
START
step 1: load data
step 2: load tokenizer
step 3: init data


Map:   0%|          | 0/1876 [00:00<?, ? examples/s]

Map:   0%|          | 0/234 [00:00<?, ? examples/s]

Map:   0%|          | 0/234 [00:00<?, ? examples/s]

step 4: load model


...

Epoch,Training Loss,Validation Loss,R2 Score,Mean Squared Error,Accuracy,F1,Precision,Recall
1,0.286500,0.265424,0.206553,0.515193,0.662393,0.342055,0.557057,0.366763
2,0.256900,0.260134,0.222367,0.510033,0.675214,0.337537,0.490830,0.371587
3,0.223300,0.229169,0.314932,0.478716,0.726496,0.520346,0.477889,0.572304
4,0.197000,0.283785,0.151666,0.532715,0.713675,0.501950,0.462963,0.549951
5,0.166100,0.192922,0.423287,0.439229,0.735043,0.659548,0.674660,0.655284
6,0.114000,0.194066,0.419869,0.440529,0.782051,0.709341,0.752736,0.691372
7,0.114700,0.176645,0.471944,0.420292,0.786325,0.696283,0.781412,0.678849
8,0.079700,0.178468,0.466497,0.422454,0.786325,0.691559,0.808666,0.641338
9,0.074900,0.168418,0.496538,0.410388,0.782051,0.696388,0.765632,0.681794
10,0.060300,0.177723,0.468723,0.421572,0.816239,0.761243,0.792593,0.750570


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


./Regressors/xl_task2_auth_usr4/checkpoint-330


step 6: evaluate


{'eval_loss': 0.24916395545005798, 'eval_r2_score': 0.25185224030628295, 'eval_mean_squared_error': 0.49916326999664307, 'eval_accuracy': 0.7478632478632479, 'eval_f1': 0.71013126009097, 'eval_precision': 0.6938915296758434, 'eval_recall': 0.7417600527356624, 'eval_runtime': 1.5424, 'eval_samples_per_second': 151.716, 'eval_steps_per_second': 2.593, 'epoch': 20.0}
DONE
==== round 5 ====
Result[val] {'eval_loss': 0.17988237738609314, 'eval_r2_score': 0.4622678787753699, 'eval_mean_squared_error': 0.42412543296813965, 'eval_accuracy': 0.8162393162393162, 'eval_f1': 0.7779953079653712, 'eval_precision': 0.7752283240088119, 'eval_recall': 0.7839998866646832, 'eval_runtime': 1.5604, 'eval_samples_per_second': 149.96, 'eval_steps_per_second': 2.563, 'epoch': 20.0}
Result[test] {'eval_loss': 0.24916395545005798, 'eval_r2_score': 0.25185224030628295, 'eval_mean_squared_error': 0.49916326999664307, 'eval_accuracy': 0.7478632478632479, 'eval_f1': 0.71013126009097, 'eval_precision': 0.69389152967

In [41]:
for i in range(5):
  src = f"./Regressors/{task_name}{i}/best_model"
  tgt = f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/Regressors/{task_name}{i}"

  if i==best_trainer_idx:
    copytree(f"{src}/train.csv", f"{tgt}/train.csv", symlinks=False, ignore=None)
    copytree(f"{src}/val.csv", f"{tgt}/val.csv", symlinks=False, ignore=None)
    copytree(f"{src}/test.csv", f"{tgt}/test.csv", symlinks=False, ignore=None)
    copytree(f"{src}/eval.json", f"{tgt}/eval.json", symlinks=False, ignore=None)

    tgt = f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/Regressors/{task_name}_best"
    copytree(src, tgt, symlinks=False, ignore=None)
  else:
    copytree(f"{src}/train.csv", f"{tgt}/train.csv", symlinks=False, ignore=None)
    copytree(f"{src}/val.csv", f"{tgt}/val.csv", symlinks=False, ignore=None)
    copytree(f"{src}/test.csv", f"{tgt}/test.csv", symlinks=False, ignore=None)
    copytree(f"{src}/eval.json", f"{tgt}/eval.json", symlinks=False, ignore=None)

  print(f"SAVED {i}")

SAVED 0
SAVED 1
SAVED 2
SAVED 3
SAVED 4


In [42]:
get_shapley(df, best_trainer, f"./ShapleyValues/{task_name}.pkl");
copytree(f"./ShapleyValues/{task_name}.pkl", f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/ShapleyValues/{task_name}.pkl", symlinks=False, ignore=None)

DATA SIZE 2344


PartitionExplainer explainer: 2345it [1:44:29,  2.68s/it]


In [43]:
!rm -rf ./Regressors/*

In [ ]:
# del best_trainer
# del _f1

## Task3: Train Model

In [72]:
best_trainer = None
best_eval_val = None
best_eval_test = None
best_trainer_idx = None

task_name = "xl_task3_clse_usr"
for i in range(5):
  df = get_task1_conver("../Task3/annotated/annotated.jsonl", "closeness", skips = ["4. Don't like each other"], only_user=True)
  trainer, eval_val, eval_test = run_exp(f"./Regressors/{task_name}{i}", df, seed=i, report=report, regressor_configs={
      "label": "close",
      "not_label": "not_close",
      "label_fn": closeness_label_fn,
  })

  print(f"==== round {i+1} ====")
  print("Result[val]", eval_val)
  print("Result[test]", eval_test)

  if abs(eval_val["eval_f1"] - eval_test["eval_f1"]) > 0.15:
    continue

  if (best_eval_val is None) or (best_eval_val["eval_f1"] < eval_val["eval_f1"]):
    best_eval_val = eval_val
    best_eval_test = eval_test
    best_trainer = trainer
    best_trainer_idx = i

  del trainer

print()
print()
print("===================================")
print()
print("BEST MODEL")
print("Result[val]", best_eval_val)
print("Result[test]", best_eval_test)


Loaded 1221 records from ../Task3/annotated/annotated.jsonl
N 1090 60 60
START
step 1: load data
step 2: load tokenizer
step 3: init data


Map:   0%|          | 0/1090 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

step 4: load model


...

./Regressors/xl_task3_clse_usr4/checkpoint-54


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


step 6: evaluate


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.1914980262517929, 'eval_r2_score': 0.3681091717755941, 'eval_mean_squared_error': 0.4376048743724823, 'eval_accuracy': 0.7166666666666667, 'eval_f1': 0.4541666666666668, 'eval_precision': 0.48611111111111116, 'eval_recall': 0.45630630630630636, 'eval_runtime': 0.3711, 'eval_samples_per_second': 161.66, 'eval_steps_per_second': 2.694, 'epoch': 20.0}
DONE
==== round 5 ====
Result[val] {'eval_loss': 0.2759150266647339, 'eval_r2_score': 0.16948656113540672, 'eval_mean_squared_error': 0.5252761244773865, 'eval_accuracy': 0.6333333333333333, 'eval_f1': 0.5564814814814815, 'eval_precision': 0.7771464646464646, 'eval_recall': 0.5198611111111111, 'eval_runtime': 0.3773, 'eval_samples_per_second': 159.022, 'eval_steps_per_second': 2.65, 'epoch': 20.0}
Result[test] {'eval_loss': 0.1914980262517929, 'eval_r2_score': 0.3681091717755941, 'eval_mean_squared_error': 0.4376048743724823, 'eval_accuracy': 0.7166666666666667, 'eval_f1': 0.4541666666666668, 'eval_precision': 0.4861111111111

In [73]:
for i in range(5):
  src = f"./Regressors/{task_name}{i}/best_model"
  tgt = f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/Regressors/{task_name}{i}"

  if i==best_trainer_idx:
    copytree(f"{src}/train.csv", f"{tgt}/train.csv", symlinks=False, ignore=None)
    copytree(f"{src}/val.csv", f"{tgt}/val.csv", symlinks=False, ignore=None)
    copytree(f"{src}/test.csv", f"{tgt}/test.csv", symlinks=False, ignore=None)
    copytree(f"{src}/eval.json", f"{tgt}/eval.json", symlinks=False, ignore=None)

    tgt = f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/Regressors/{task_name}_best"
    copytree(src, tgt, symlinks=False, ignore=None)
  else:
    copytree(f"{src}/train.csv", f"{tgt}/train.csv", symlinks=False, ignore=None)
    copytree(f"{src}/val.csv", f"{tgt}/val.csv", symlinks=False, ignore=None)
    copytree(f"{src}/test.csv", f"{tgt}/test.csv", symlinks=False, ignore=None)
    copytree(f"{src}/eval.json", f"{tgt}/eval.json", symlinks=False, ignore=None)

  print(f"SAVED {i}")

SAVED 0
SAVED 1
SAVED 2
SAVED 3
SAVED 4


In [74]:
get_shapley(df, best_trainer, f"./ShapleyValues/{task_name}.pkl");
copytree(f"./ShapleyValues/{task_name}.pkl", f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/ShapleyValues/{task_name}.pkl", symlinks=False, ignore=None)

DATA SIZE 1210


PartitionExplainer explainer: 1211it [1:13:47,  3.67s/it]


In [75]:
del best_trainer
# del _f1

In [29]:
best_trainer = None
best_eval_val = None
best_eval_test = None
best_trainer_idx = None

task_name = "xl_task3_auth_usr"
for i in range(5):
  df = get_task1_conver("../Task3/annotated/annotated.jsonl", "authority", skips = [], only_user=True)
  trainer, eval_test, eval_val = run_exp(f"./Regressors/{task_name}{i}", df, seed=i, report=report, regressor_configs={
      "label": "respect",
      "not_label": "not_respect",
      "label_fn": authority2_label_fn,
  })

  print(f"==== round {i+1} ====")
  print("Result[val]", eval_val)
  print("Result[test]", eval_test)

  if abs(eval_val["eval_f1"] - eval_test["eval_f1"]) > 0.15:
    del trainer
    continue

  if (best_eval_val is None) or (best_eval_val["eval_f1"] < eval_val["eval_f1"]):
    best_eval_val = eval_val
    best_eval_test = eval_test
    best_trainer = trainer
    best_trainer_idx = i

  del trainer

print()
print()
print("===================================")
print()
print("BEST MODEL")
print("Result[val]", best_eval_val)
print("Result[test]", best_eval_test)


Loaded 1221 records from ../Task3/annotated/annotated.jsonl
N 1099 61 61
START
step 1: load data
step 2: load tokenizer


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


step 3: init data


Map:   0%|          | 0/1099 [00:00<?, ? examples/s]

Map:   0%|          | 0/61 [00:00<?, ? examples/s]

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


step 6: evaluate


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.09131775051355362, 'eval_r2_score': 0.15474289963972565, 'eval_mean_squared_error': 0.30218827724456787, 'eval_accuracy': 0.8852459016393442, 'eval_f1': 0.31304347826086953, 'eval_precision': 0.29508196721311475, 'eval_recall': 0.3333333333333333, 'eval_runtime': 0.4156, 'eval_samples_per_second': 146.789, 'eval_steps_per_second': 2.406, 'epoch': 20.0}
DONE
==== round 5 ====
Result[val] {'eval_loss': 0.09131775051355362, 'eval_r2_score': 0.15474289963972565, 'eval_mean_squared_error': 0.30218827724456787, 'eval_accuracy': 0.8852459016393442, 'eval_f1': 0.31304347826086953, 'eval_precision': 0.29508196721311475, 'eval_recall': 0.3333333333333333, 'eval_runtime': 0.4156, 'eval_samples_per_second': 146.789, 'eval_steps_per_second': 2.406, 'epoch': 20.0}
Result[test] {'eval_loss': 0.14331023395061493, 'eval_r2_score': 0.12003735967920048, 'eval_mean_squared_error': 0.37856337428092957, 'eval_accuracy': 0.8524590163934426, 'eval_f1': 0.40154440154440146, 'eval_precision': 0.

In [30]:
for i in range(5):
  src = f"./Regressors/{task_name}{i}/best_model"
  tgt = f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/Regressors/{task_name}{i}"

  if i==best_trainer_idx:
    copytree(f"{src}/train.csv", f"{tgt}/train.csv", symlinks=False, ignore=None)
    copytree(f"{src}/val.csv", f"{tgt}/val.csv", symlinks=False, ignore=None)
    copytree(f"{src}/test.csv", f"{tgt}/test.csv", symlinks=False, ignore=None)
    copytree(f"{src}/eval.json", f"{tgt}/eval.json", symlinks=False, ignore=None)

    tgt = f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/Regressors/{task_name}_best"
    copytree(src, tgt, symlinks=False, ignore=None)
  else:
    copytree(f"{src}/train.csv", f"{tgt}/train.csv", symlinks=False, ignore=None)
    copytree(f"{src}/val.csv", f"{tgt}/val.csv", symlinks=False, ignore=None)
    copytree(f"{src}/test.csv", f"{tgt}/test.csv", symlinks=False, ignore=None)
    copytree(f"{src}/eval.json", f"{tgt}/eval.json", symlinks=False, ignore=None)

  print(f"SAVED {i}")

SAVED 0
SAVED 1
SAVED 2
SAVED 3
SAVED 4


In [34]:

get_shapley(df, best_trainer, f"./ShapleyValues/{task_name}.pkl");
copytree(f"./ShapleyValues/{task_name}.pkl", f"/content/drive/MyDrive/TalkLikeMom/src/Classifier/ShapleyValues/{task_name}.pkl", symlinks=False, ignore=None)

DATA SIZE 1221


PartitionExplainer explainer: 1222it [1:15:36,  3.72s/it]


In [ ]:
!rm -rf ./Regressors/*

In [ ]:
del best_trainer
del _f1